In [1]:
# Prepare UniXcoder source file
import os
from urllib import request
unixcoder_url = 'https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py'

if not (os.path.exists('./unixcoder.py')):
    with request.urlopen(unixcoder_url) as response, open('unixcoder.py', 'wb') as out_file:
        data = response.read()
        out_file.write(data)

In [2]:
import torch    
from unixcoder import UniXcoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UniXcoder("microsoft/unixcoder-base")
model.to(device)

UniXcoder(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(51416, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768, padding_idx=1)
      (token_type_embeddings): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,),

# Example: Encoder-only Mode

For encoder-only mode, we give an example of code search.

## 1) Code and NL Embeddings

Here, we give an example to obtain code fragment embedding from CodeBERT.

In [12]:
# Encode maximum function
func = "def f(a,b): if a>b: return a else return b"
tokens_ids = model.tokenize([func],max_length=512,mode="<encoder-only>")
source_ids = torch.tensor(tokens_ids).to(device)
tokens_embeddings,max_func_embedding = model(source_ids)

# Encode minimum function
func = "def f(a,b): if a<b: return a else return b"
tokens_ids = model.tokenize([func],max_length=512,mode="<encoder-only>")
source_ids = torch.tensor(tokens_ids).to(device)
tokens_embeddings,min_func_embedding = model(source_ids)

# Encode NL
nl = "return maximum value"
tokens_ids = model.tokenize([nl],max_length=512,mode="<encoder-only>")
source_ids = torch.tensor(tokens_ids).to(device)
tokens_embeddings,nl_embedding = model(source_ids)

print(max_func_embedding.shape)
print(max_func_embedding)

torch.Size([1, 768])
tensor([[ 8.6533e-01, -1.9796e+00, -8.6849e-01,  4.2652e-01, -5.3695e-01,
         -1.5521e-01,  5.3771e-01,  3.4200e-01,  3.6306e-01, -3.9392e-01,
         -1.1816e+00,  2.6010e+00, -7.7133e-01,  1.8441e+00,  2.3645e+00,
         -8.0971e-01,  1.9955e+00,  6.4072e-01, -2.4817e-01, -2.6337e+00,
         -1.1199e+00, -2.3594e+00, -7.5279e-01,  1.1628e-01,  1.6286e+00,
         -4.5879e-02, -1.4457e+00,  7.8070e-01,  7.7836e-01, -5.6094e-01,
          2.2112e+00, -7.8929e-01, -2.3475e+00,  1.7342e+00, -8.8963e-01,
         -6.9369e-01,  2.4841e+00,  4.9074e-01, -8.5599e-01,  1.0941e-01,
         -1.4604e+00,  2.3402e+00, -1.8161e+00, -2.9290e+00, -3.9770e+00,
         -4.7137e-01,  1.7589e+00,  2.5646e+00, -4.2621e-01, -8.0802e-02,
         -2.9151e+00,  1.6699e+00, -1.9765e+00, -8.8960e-01,  8.9094e-01,
         -6.7859e-01, -4.2710e-01, -2.1378e+00, -2.3407e+00, -4.9259e-01,
          1.6661e+00, -1.4553e+00, -4.7143e-01,  1.7414e+00, -1.4021e+00,
         -3.6193e

## 2) Similarity between code and NL

Now, we calculate cosine similarity between NL and two functions. Although the difference of two functions is only a operator (< and >), UniXcoder can distinguish them.

In [14]:
# Normalize embedding
norm_max_func_embedding = torch.nn.functional.normalize(max_func_embedding, p=2, dim=1)
norm_min_func_embedding = torch.nn.functional.normalize(min_func_embedding, p=2, dim=1)
norm_nl_embedding = torch.nn.functional.normalize(nl_embedding, p=2, dim=1)
print(norm_max_func_embedding.shape)

max_func_nl_similarity = torch.einsum("ac,bc->ab",norm_max_func_embedding,norm_nl_embedding)
min_func_nl_similarity = torch.einsum("ac,bc->ab",norm_min_func_embedding,norm_nl_embedding)

print(max_func_nl_similarity)
print(min_func_nl_similarity)

torch.Size([1, 768])
tensor([[0.3002]], grad_fn=<ViewBackward0>)
tensor([[0.1881]], grad_fn=<ViewBackward0>)


# 3. Decoder-only Mode

For decoder-only mode, we give an example of **code completion**.

In [ ]:
context = """
def f(data,file_path):
    # write json data into file_path in python language
"""
tokens_ids = model.tokenize([context],max_length=512,mode="<decoder-only>")
source_ids = torch.tensor(tokens_ids).to(device)
prediction_ids = model.generate(source_ids, decoder_only=True, beam_size=3, max_length=128)
predictions = model.decode(prediction_ids)
print(context+predictions[0][0])

# 4. Encoder-Decoder Mode

For encoder-decoder mode, we give two examples including: function name prediction, API recommendation, code summarization.

## 1) Function Name Prediction

In [ ]:
context = """
def <mask0>(data,file_path):
    data = json.dumps(data)
    with open(file_path, 'w') as f:
        f.write(data)
"""
tokens_ids = model.tokenize([context],max_length=512,mode="<encoder-decoder>")
source_ids = torch.tensor(tokens_ids).to(device)
prediction_ids = model.generate(source_ids, decoder_only=False, beam_size=3, max_length=128)
predictions = model.decode(prediction_ids)
print([x.replace("<mask0>","").strip() for x in predictions[0]])

In [ ]:
# slothPete7773's example
context = """
def <mask0>(url, file_path):
    data = request.urlopen(url)
    data = data.read()
    with open(file_path, 'w') as f:
        f.write(data)
"""
tokens_ids = model.tokenize([context],max_length=512,mode="<encoder-decoder>")
source_ids = torch.tensor(tokens_ids).to(device)
prediction_ids = model.generate(source_ids, decoder_only=False, beam_size=3, max_length=128)
predictions = model.decode(prediction_ids)
print([x.replace("<mask0>","").strip() for x in predictions[0]])

## 2) API Recommendation

In [ ]:
context = """
def write_json(data,file_path):
    data = <mask0>(data)
    with open(file_path, 'w') as f:
        f.write(data)
"""
tokens_ids = model.tokenize([context],max_length=512,mode="<encoder-decoder>")
source_ids = torch.tensor(tokens_ids).to(device)
prediction_ids = model.generate(source_ids, decoder_only=False, beam_size=3, max_length=128)
predictions = model.decode(prediction_ids)
print([x.replace("<mask0>","").strip() for x in predictions[0]])

## 3) Code Summarization

In [4]:
context = """
# <mask0>
def write_json(data,file_path):
    data = json.dumps(data)
    with open(file_path, 'w') as f:
        f.write(data)
"""
tokens_ids = model.tokenize([context],max_length=512,mode="<encoder-decoder>")
print(tokens_ids)

[[0, 5, 2, 317, 121, 19, 729, 2250, 181, 2317, 126, 636, 130, 824, 181, 902, 953, 317, 377, 869, 385, 3192, 132, 17284, 126, 636, 127, 317, 377, 918, 2717, 126, 824, 181, 902, 130, 464, 205, 784, 880, 412, 144, 317, 394, 412, 132, 1112, 126, 636, 127, 317, 2]]


In [5]:
source_ids = torch.tensor(tokens_ids).to(device)
print(source_ids)

tensor([[    0,     5,     2,   317,   121,    19,   729,  2250,   181,  2317,
           126,   636,   130,   824,   181,   902,   953,   317,   377,   869,
           385,  3192,   132, 17284,   126,   636,   127,   317,   377,   918,
          2717,   126,   824,   181,   902,   130,   464,   205,   784,   880,
           412,   144,   317,   394,   412,   132,  1112,   126,   636,   127,
           317,     2]])


In [9]:
prediction_ids = model.generate(source_ids, decoder_only=False, beam_size=3, max_length=128)
print(prediction_ids.shape)
print(prediction_ids)

torch.Size([1, 3, 128])
tensor([[[  19, 3813, 3798,  508, 1012,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0],
         [  19, 3813, 3192,  508, 1012,    0,    0,    0,    0,    0,    0,
           

In [11]:
predictions = model.decode(prediction_ids)
# print(predictions.shape)
print(predictions)

[['<mask0> Write JSON to file', '<mask0> Write json to file', '<mask0> Write a json file']]


In [8]:
print([x.replace("<mask0>","").strip() for x in predictions[0]])

['Write JSON to file', 'Write json to file', 'Write a json file']
